In [1]:
!pip install kaleido
!pip install skforecast
!pip install ordpy
!pip install EntropyHub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 692.3/692.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 1.8 MB/s eta 0:00:00


In [3]:
import os
import pandas as pd
import numpy as np
from skforecast.model_selection import backtesting_forecaster
from sklearn.linear_model import LinearRegression
from skforecast.ForecasterAutoreg import ForecasterAutoreg
import ordpy
import plotly.graph_objects as go
import EntropyHub as EH
import tqdm as tqdm
import statsmodels.api as sm
import kaleido
import plotly.io as pio
import plotly.offline as poff
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
from skforecast.ForecasterBaseline import ForecasterEquivalentDate
from skforecast.model_selection import backtesting_forecaster

ModuleNotFoundError: No module named 'skforecast.ForecasterAutoreg'

In [5]:
# hay que conectarse al drive en cada sesión
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [6]:
RUTA_DATA = "/content/drive/MyDrive/Proyecto_UTN_FRBA_Becarios/Códigos/2024/01 - Pronostico de demanda (experimento 1)/Datasets/"

data = pd.read_csv(os.path.join(RUTA_DATA, "dataset.csv"), index_col='fecha', parse_dates=['fecha'])
data = data.resample(rule='H', closed='left', label ='right').mean()
data.temp = data.temp.ffill()
data.tail()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Proyecto_UTN_FRBA_Becarios/Códigos/2024/01 - Pronostico de demanda (experimento 1)/Datasets/dataset.csv'

In [ ]:
# se definen los intervalos de entranamiento y pronostico
end_train = '2023-12-31 23:59:00'
data_train = data.loc[:end_train, :].copy()
data_test = data.loc[end_train:, :].copy()

print(f"Train dates      : {data_train.index.min()} --- {data_train.index.max()}  (n={len(data_train)})")
print(f"Test dates       : {data_test.index.min()} --- {data_test.index.max()}  (n={len(data_test)})")

Train dates      : 2021-01-01 03:00:00 --- 2023-12-31 23:00:00  (n=26277)
Test dates       : 2024-01-01 00:00:00 --- 2024-02-01 01:00:00  (n=746)


# MODELO BASE

In [ ]:
# se selecciona el pronosticador
forecaster = ForecasterEquivalentDate(
                 offset    = pd.DateOffset(days=1),
                 n_offsets = 1
             )
metric, predictions = backtesting_forecaster(
                          forecaster         = forecaster,
                          y                  = data['demanda'],
                          steps              = 24,
                          metric             = 'mean_squared_error',
                          initial_train_size = len(data.loc[:end_train]),
                          refit              = False,
                          n_jobs             = 'auto',
                          verbose            = False,
                          show_progress      = True
                      )

print(f"MSE: {metric:.2f}")
MODELO_BASE = metric

  0%|          | 0/32 [00:00<?, ?it/s]

MSE: 3119371.47


# MODELO 1 (LINEAL)

In [ ]:
forecaster = ForecasterAutoreg(
    regressor=LinearRegression(),   # que modelo se usa
    lags=24*7                       # cuanto mira hacia atras
)
metric, predictions = backtesting_forecaster(
                          forecaster         = forecaster,
                          y                  = data['demanda'],
                          steps              = 24,
                          metric             = 'mean_squared_error',
                          initial_train_size = len(data.loc[:end_train]),
                          refit              = False,
                          n_jobs             = 'auto',
                          verbose            = False,
                          show_progress      = True
                      )

print(f"MSE: {metric:.2f}")
MODELO_1 = metric

  0%|          | 0/32 [00:00<?, ?it/s]

MSE: 1530491.74


# MODELO CON VARIABLES EXOGENAS
Se prueban todas las posibles combinaciones

In [ ]:
# Renombro temperatura
data["T"] = data["temp"]

# calculo entropia H
entropia = [0] * 24*7
complejidad = [0] * 24*7
for i in range(24*7, len(data.demanda), 1):
    h, c = ordpy.complexity_entropy(data.demanda[i-24*7:i], dx=3)
    entropia.append(h)
    complejidad.append(c)
data['H'] = entropia

# calcula demanda estimada DE1
data["DE"] = data["T"].map(lambda t: 31.9*t**2 - 1081*t + 23620)

# dia habil o no
data["DH"] = data["no_habil"]

In [ ]:

import itertools
variables_exogenas = ["T", "H", "DE", "DH"]
combinaciones = []
for i in range(1, len(variables_exogenas) + 1):
    combinaciones += list(itertools.combinations(variables_exogenas, i))
combinaciones = [list(x) for x in combinaciones]
print(combinaciones)


[['T'], ['H'], ['DE'], ['DH'], ['T', 'H'], ['T', 'DE'], ['T', 'DH'], ['H', 'DE'], ['H', 'DH'], ['DE', 'DH'], ['T', 'H', 'DE'], ['T', 'H', 'DH'], ['T', 'DE', 'DH'], ['H', 'DE', 'DH'], ['T', 'H', 'DE', 'DH']]


In [ ]:
for combinacion in combinaciones:
    forecaster = ForecasterAutoreg(
        regressor=LinearRegression(),   # que modelo se usa
        lags=24*7                       # cuanto mira hacia atras
    )
    metric, predictions = backtesting_forecaster(
                            forecaster         = forecaster,
                            y                  = data['demanda'],
                            exog               = data[combinacion],
                            steps              = 24,
                            metric             = 'mean_squared_error',
                            initial_train_size = len(data.loc[:end_train]),
                            refit              = False,
                            n_jobs             = 'auto',
                            verbose            = False,
                            show_progress      = True
                        )
    variables = '\t'.join(combinacion)
    print(f"{metric:.2f}\t{variables}")


  0%|          | 0/32 [00:00<?, ?it/s]

1476649.45	T


  0%|          | 0/32 [00:00<?, ?it/s]

1545521.44	H


  0%|          | 0/32 [00:00<?, ?it/s]

...

1246617.62	T	DE	DH


  0%|          | 0/32 [00:00<?, ?it/s]

1243526.69	H	DE	DH


  0%|          | 0/32 [00:00<?, ?it/s]

1247569.37	T	H	DE	DH
